In [131]:
#dependencies
import pandas as pd
import numpy as np

In [132]:
#import csv path
csv_path_schools = "schools_complete.csv"
csv_path_students = "students_complete.csv"

In [133]:
#read in csv files
schools_df = pd.read_csv(csv_path_schools)
students_df = pd.read_csv(csv_path_students)

In [134]:
schools_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [227]:
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score,read_pass,math_pass
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,1
1,1,Victor Smith,M,12th,Huang High School,94,61,1,0
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,1,0
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0
4,4,Bonnie Ray,F,9th,Huang High School,97,84,1,1


In [228]:
#make count for passing scores in new column
students_df['read_pass'] = np.where(students_df['reading_score']>=70, 1, 0)
students_df['math_pass'] = np.where(students_df['math_score']>=70, 1, 0)
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score,read_pass,math_pass
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,1
1,1,Victor Smith,M,12th,Huang High School,94,61,1,0
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,1,0
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0
4,4,Bonnie Ray,F,9th,Huang High School,97,84,1,1


In [145]:
students_df.columns

Index(['Student ID', 'name', 'gender', 'grade', 'school', 'reading_score',
       'math_score', 'read_pass', 'math_pass'],
      dtype='object')

In [158]:
#group student data by school & apply sum method
groupbyschool_students_sum = students_df.groupby('school', as_index=False).sum()
groupbyschool_students_sum[['school','read_pass', 'math_pass']].head()

,school,read_pass,math_pass
0,Bailey High School,4077,3318
1,Cabrera High School,1803,1749
2,Figueroa High School,2381,1946
3,Ford High School,2172,1871
4,Griffin High School,1426,1371


In [148]:
groupbyschool_students_sum.columns

Index(['school', 'Student ID', 'reading_score', 'math_score'], dtype='object')

In [173]:
#group student data by school & apply mean method
groupbyschool_students_mean = students_df.groupby('school', as_index=False).mean()
average_scores = groupbyschool_students_mean[['school', 'reading_score', 'math_score']]

#rename columns
average_scores.rename(columns = {'reading_score':'Average Reading Score', 'math_score':'Average Math Score'}, inplace = True)
average_scores.head()

/Users/stefanirobnett/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,school,Average Reading Score,Average Math Score
0,Bailey High School,81.033963,77.048432
1,Cabrera High School,83.975780,83.061895
2,Figueroa High School,81.158020,76.711767
3,Ford High School,80.746258,77.102592
4,Griffin High School,83.816757,83.351499


In [179]:
#merge student_mean & student_sum data 
merged_students_sum_average = groupbyschool_students_sum.merge(average_scores, on='school')
#Re-assign groupby to df
#Omit unnecessary columns(reading_score, math_score, student id)
students_sum_average = merged_students_sum_average[['school', 'Average Reading Score', 'Average Math Score', 'read_pass', 'math_pass']]
students_sum_average.head()

,school,Average Reading Score,Average Math Score,read_pass,math_pass
0,Bailey High School,81.033963,77.048432,4077,3318
1,Cabrera High School,83.975780,83.061895,1803,1749
2,Figueroa High School,81.158020,76.711767,2381,1946
3,Ford High School,80.746258,77.102592,2172,1871
4,Griffin High School,83.816757,83.351499,1426,1371


In [332]:
#merge groubyschool_student_df with school_df
#re-assign chart name as "School Summary"
school_summary = pd.merge(schools_df, students_sum_average, left_on='name', right_on='school')
#Rename columns
school_summary.rename(columns = {'school':'School Name', 'type':'School Type','size':'Total Students', 'budget': 'Total School Budget'}, inplace = True)

In [333]:
#divide read_pass by size to get percent passed
school_summary['% Passing Reading'] = school_summary['read_pass']/school_summary['Total Students']*100
#divide math_pass by size to get percent passed
school_summary['% Passing Math'] = school_summary['math_pass']/school_summary['Total Students']*100
#divide budget by size for budget per student
school_summary['Per Student Budget'] = school_summary['Total School Budget']/school_summary['Total Students']
#average of read_pass and Math_pass overall passing
school_summary['Overall Passing Rate'] = school_summary[['% Passing Reading', '% Passing Math']].mean(axis=1)

In [334]:
#Omit un. columns
school_summary= school_summary[['School Name', 'School Type', 
                                'Total Students', 'Total School Budget', 
                                'Per Student Budget', 'Average Math Score', 
                                'Average Reading Score', '% Passing Math', 
                                '% Passing Reading', 'Overall Passing Rate']]
school_summary

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


In [335]:
#Top Performing Schools (by passing rate)
#Get top 5 schools
top_five_schools = school_summary.nlargest(5, 'Overall Passing Rate')
top_five_schools

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
14,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [336]:
#Top Performing Schools (by passing rate)
#Get bottom 5 schools
bottom_five_schools = school_summary.nsmallest(5, 'Overall Passing Rate')
bottom_five_schools

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
13,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [338]:
#Group by schools and Grade
# Apply mean function for reading & math scores
students_df_grade_mean = students_df.groupby(['school','grade'], as_index=False).mean()
#rename columns
students_df_grade_mean.rename(columns= {'school':'School Name', 'grade': 'Grade','reading_score':'Average Reading Score', 'math_score':'Average Math Score'}, inplace = True)


In [340]:
#Create Reading Scores By Grade
reading_scores_by_grade = students_df_grade_mean[['School Name', 'Grade', 'Average Reading Score']]
reading_scores_by_grade

,School Name,Grade,Average Reading Score
0,Bailey High School,10th,80.907183
1,Bailey High School,11th,80.945643
2,Bailey High School,12th,80.912451
3,Bailey High School,9th,81.303155
4,Cabrera High School,10th,84.253219
5,Cabrera High School,11th,83.788382
6,Cabrera High School,12th,84.287958
7,Cabrera High School,9th,83.676136
8,Figueroa High School,10th,81.408912
9,Figueroa High School,11th,80.640339


In [339]:
#Create Math Scores By Grade
math_scores_by_grade = students_df_grade_mean[['School Name', 'Grade', 'Average Math Score']]
math_scores_by_grade

,School Name,Grade,Average Math Score
0,Bailey High School,10th,76.996772
1,Bailey High School,11th,77.515588
2,Bailey High School,12th,76.492218
3,Bailey High School,9th,77.083676
4,Cabrera High School,10th,83.154506
5,Cabrera High School,11th,82.765560
6,Cabrera High School,12th,83.277487
7,Cabrera High School,9th,83.094697
8,Figueroa High School,10th,76.539974
9,Figueroa High School,11th,76.884344


In [341]:
#Groupby spending ranges (per student):
#Create copy of school_summary for spending ranges
school_summary_spending_ranges = school_summary
#Create bins and bin labels Average Spending Ranges per student to school_summary
school_summary_spending_ranges["Spending Ranges (Per Student)"] = pd.qcut(school_summary_spending_ranges['Per Student Budget'], 4, labels=["High", "Medium-high", "Medium-low", "Low"])
#Omit unnecessary columns
school_summary_spending_ranges = school_summary_spending_ranges[['Spending Ranges (Per Student)', 'School Name', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]
#Groupby School Name and Spending Ranges (per student)
school_summary_spending_ranges_grouped = school_summary_spending_ranges.groupby(['Spending Ranges (Per Student)'], as_index=False).mean()
school_summary_spending_ranges_grouped

,Spending Ranges (Per Student),Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,High,83.455399,83.933814,93.460096,96.610877,95.035486
1,Medium-high,81.899826,83.155286,87.133538,92.718205,89.925871
2,Medium-low,78.990942,81.917212,75.209078,86.089386,80.649232
3,Low,77.023555,80.957446,66.701010,80.675217,73.688113


In [342]:
#Groupby school size:
#Create copy of school_summary for school size
school_summary_size_ranges = school_summary
#Create bins and bin labels Average Spending Ranges per student to school_summary
school_summary_size_ranges["School Size"] = pd.qcut(school_summary_size_ranges['Total Students'], 3, labels=["Large", "Medium", "Small"])
#Omit unnecessary columns
school_summary_size_ranges = school_summary_size_ranges[['School Size', 'School Name', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]
#Group by school size
school_summary_size_ranges_grouped = school_summary_size_ranges.groupby(['School Size'], as_index=False).mean()
school_summary_size_ranges_grouped

,School Size,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Large,83.554500,83.850220,93.526422,96.500267,95.013345
1,Medium,80.750065,82.769850,83.065610,90.161203,86.613407
2,Small,76.993025,80.967495,66.369129,80.995600,73.682364


In [343]:
#Groupby school type: 
#Create copy of school_summary for school type
school_summary_type = school_summary
#Create bins and bin labels Average Spending Ranges per student to school_summary
#school_summary_type["School Size"] = pd.qcut(school_summary_type['Total Students'], 3, labels=["Large", "Medium", "Small"])
#Omit unnecessary columns
school_summary_type = school_summary_type[['School Type', 'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', 'Overall Passing Rate']]
#Groupby Type
school_summary_type_grouped = school_summary_type.groupby(['School Type'], as_index=False).mean()
school_summary_type_grouped

,School Type,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Charter,83.473852,83.896421,93.620830,96.586489,95.103660
1,District,76.956733,80.966636,66.548453,80.799062,73.673757


In [331]:
#District Summary:
#Find total number of schools
school_count = len(school_summary.index)

#apply sum function
district_school_summary_sum = school_summary[['Total Students', 'Total School Budget']].sum()
district_school_summary_sum.head()
#apply mean function
district_school_summary_mean = school_summary[['Average Math Score', 'Average Reading Score','% Passing Math', '% Passing Reading', 'Overall Passing Rate']].mean()
district_school_summary_mean.head()
#combine series 
district_summary = pd.concat([district_school_summary_mean, district_school_summary_sum, pd.Series({'Total Schools': school_count})])
district_summary

Average Math Score       8.043253e+01
Average Reading Score    8.252919e+01
% Passing Math           8.098705e+01
% Passing Reading        8.921902e+01
Overall Passing Rate     8.510304e+01
Total Students           3.917000e+04
Total School Budget      2.464943e+07
Total Schools            1.500000e+01
dtype: float64